In [0]:
-- table names
SELECT table_name
FROM information_schema.tables
WHERE table_catalog = 'test_catalog';

-- table access
SELECT 
  grantee,
  table_name,
  privilege_type
FROM table_privileges;

-- table edits
SELECT
  table_name,
  last_altered_by,
  last_altered
FROM information_schema.tables
WHERE table_schema = 'bronze'
ORDER BY 1, 3 DESC;

-- owner
SELECT table_owner
FROM system.information_schema.tables
WHERE table_name = 'bronze';

-- access patterns
SELECT
  user_identity.email,
  count(*)
FROM system.access.audit
GROUP BY 1
ORDER BY 2 DESC;

-- delete
SELECT
  user_identity.email
FROM system.access.audit
WHERE action_name = "delete";

-- timebased access
SELECT
  request_params.table_full_name,
  request_params.operation,
  event_date
FROM system.access.audit
WHERE action_name = "generateTemporaryTableCredential" AND
      service_name = "unityCatalog" AND
      date_diff(now(), event_time) < 60 AND
      user_identity.email = "adm-janne.eskola@finavia.fi"
GROUP BY
  request_params.table_full_name,
  request_params.operation,
  event_date
ORDER BY event_date DESC
;

-- most queried by user
SELECT
  request_params.table_full_name,
  count(*)
FROM system.access.audit
WHERE user_identity.email = "adm-janne.eskola@finavia.fi" AND
      action_name = "generateTemporaryTableCredential" AND
      service_name = "unityCatalog"
GROUP BY 1
ORDER BY 2 DESC
LIMIT 5;

--bronze lineage
SELECT DISTINCT target_table_full_name
FROM system.access.table_lineage
WHERE source_table_full_name = 'bronze.finavia_flights.diff'
;

-- queries to table
SELECT
  DISTINCT entity_type,
  entity_id,
  source_table_full_name
FROM system.access.table_lineage
WHERE source_table_full_name = 'bronze.finavia_flights.diff';